---
# Visualization using maps in Python

---

Another perfect tool for vizualisation data processed in Python is `folium`. It builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the Leaflet.js library. It manipulate your data in Python, then visualize it in on a Leaflet map. It enables both the binding of data to a map for choropleth visualizations as well as passing Vincent/Vega visualizations as markers on the map.

In [7]:
!sudo pip3 install -U folium

The directory '/home/ubuntu/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/ubuntu/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
Requirement already up-to-date: folium in /usr/local/lib/python3.5/dist-packages
Requirement already up-to-date: six in /usr/lib/python3/dist-packages (from folium)
Requirement already up-to-date: Jinja2 in /usr/local/lib/python3.5/dist-packages (from folium)
Requirement already up-to-date: branca in /usr/local/lib/python3.5/dist-packages (from folium)
Requirement already up-to-date: MarkupSafe>=0.23 in /usr/local/lib/python3.5/dist-packages (from Jinja2->folium)


`folium` provides very detailed map and we may use it to visualize geodata localized in a small scale. Let's get the data from the [Citibike API](http://www.citibikenyc.com/stations/json):

In [8]:
import requests
import pandas as pd

url = 'http://www.citibikenyc.com/stations/json'
results = requests.get(url).json()
data = results["stationBeanList"]

citibike = pd.DataFrame(data)

Citibike dataset contains 666 rows


,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,,6,33,,72,,2017-05-31 06:00:23 PM,40.767272,,-73.993929,,W 52 St & 11 Ave,,W 52 St & 11 Ave,1,In Service,False,39
1,,17,14,,79,,2017-05-31 05:59:27 PM,40.719116,,-74.006667,,Franklin St & W Broadway,,Franklin St & W Broadway,1,In Service,False,33
2,,15,11,,82,,2017-05-31 06:00:34 PM,40.711174,,-74.000165,,St James Pl & Pearl St,,St James Pl & Pearl St,1,In Service,False,27
3,,45,15,,83,,2017-05-31 05:58:59 PM,40.683826,,-73.976323,,Atlantic Ave & Fort Greene Pl,,Atlantic Ave & Fort Greene Pl,1,In Service,False,62
4,,0,37,,116,,2017-05-31 06:00:41 PM,40.741776,,-74.001497,,W 17 St & 8 Ave,,W 17 St & 8 Ave,1,In Service,False,39


In [25]:
import folium
fmap = folium.Map(location=[40.73, -74], zoom_start=13,  tiles='cartodbpositron')

In [26]:
for name, row in citibike.iterrows():
    
    # Define the opacity of the marker to be proportional to the percentage of bikes in the station
    opacity = row["availableBikes"]/row["totalDocks"] if row["statusValue"] == 'In Service' else 1.0
    # Make the color green for the working stations, red otherwise
    color = "green" if row["statusValue"] == 'In Service' else "red"
    # The size of the marker is proportional to the number of docks
    size = row["totalDocks"]/5 if row["statusValue"] == 'In Service' else 5
    
    # The code below defines a pop-up for each station with details such as 
    # the address, number of bikes, capacity, etc.
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Address: </strong>" + row["stAddress1"] + \
           "<br><strong>Available Bikes: </strong>" + str(row["availableBikes"]) + \
            "<br><strong>Total Docks: </strong>" + str(row["totalDocks"])
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)

    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["latitude"], row["longitude"]], 
                        radius = size,
                        popup = popup, 
                        fill_opacity = opacity,
                        fill_color = color,
                       ).add_to(fmap)

In [27]:
fmap

In [28]:
import folium
fmap = folium.Map(location=[40.73, -74], zoom_start=11, tiles='OpenStreetMap')

folium.GeoJson(open('../6-Pandas/data/nyc-neighborhoods.geojson'),
               name='NYC Neighborhoods'
              ).add_to(fmap)

folium.LayerControl().add_to(fmap)

fmap

TypeError: __init__() got an unexpected keyword argument 'weight'